<a href="https://colab.research.google.com/github/lblum95/AML/blob/master/task_1-Jannik" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AML first project

## Data ingestion

### Connect to drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')

Mounted at /content/drive


### Import all data libraries

In [2]:
!pip install pandas_profiling -U
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np

     |████████████████████████████████| 266kB 1.8MB/s 
     |████████████████████████████████| 604kB 5.0MB/s 
     |████████████████████████████████| 3.1MB 8.6MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 296kB 19.9MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27085 sha256=84dde8b361c061bcec26d434efc67f4a813878384785003ef89fa1201c9c8036
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for imagehash: filename=ImageHash-4.1.0-py2.py3-none-any.whl size=291991 sha256=df1a0e7ab3328cec4ba5a7d58ff0b3f137bdcecff3c06bcaa4fb61409c0dd1b8
  Stored in directory: /root/.cache/pip/wheels/07/1c/dc/6831446f09feb8cc199ec73a0f2f0703253f6ae013a22f4be9
Successfully built htmlmin imagehash
  Found existing installation: tqdm 4.41.1
    Unin

### Import data

In [3]:
x_train = pd.read_csv("data/X_train.csv",index_col=0,header = 0)
y_train = pd.read_csv("data/y_train.csv",index_col=0,header = 0)
x_test = pd.read_csv("data/X_test.csv",index_col=0,header = 0)

### Data cleaning


Outlier clipping

quantile ignores nans :)

In [4]:
percentile=0.05
#quantile ignores nans :)
print(x_train.shape)
lower=x_train.quantile(percentile, axis=0, interpolation='lower')
higher=x_train.quantile(1-percentile, axis=0, interpolation='higher')
print(lower.shape)
print(higher.shape)
train=x_train.clip(lower,higher,axis=1)
print(train.shape)

(1212, 832)
(832,)
(832,)
(1212, 832)




Fill mean in NaNs

In [5]:
#fill NaNs with mean
train=train.apply(lambda x: x.fillna(x.mean()),axis=0)

Normalize data

In [7]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(train)
test=scaler.transform(train)
test

array([[ 1.03395957e+00, -1.72388823e+00,  6.95963777e-01, ...,
        -5.24358916e-01, -1.71844251e+00, -7.03238121e-01],
       [ 1.77519194e+00,  1.45780409e+00,  3.96418573e-10, ...,
        -1.62112772e+00, -4.88124515e-01,  9.97882048e-02],
       [ 1.22526942e+00,  1.39000401e+00, -4.38707639e-01, ...,
        -1.13899489e+00, -1.83947835e+00, -2.81512241e-01],
       ...,
       [-9.04759519e-01, -1.63401639e+00,  3.23005914e-02, ...,
        -1.59444405e+00,  1.81870577e+00,  1.30501687e-01],
       [-1.05993919e-14, -1.22064009e+00,  1.42144559e+00, ...,
        -8.71659095e-02,  5.86512874e-01, -1.68631832e+00],
       [-1.13927439e+00, -6.58484639e-01,  2.53751452e-01, ...,
        -7.64400524e-01, -1.72872580e-14, -1.76454689e-01]])

## Feature selection

### import all libraries

In [8]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

### Fiddle down to 200 features

In [10]:
kbest=SelectKBest(f_regression, k=200)
train = kbest.fit_transform(test, y_train)
print(train.shape)

(1212, 200)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


### RFECV with linear SVR 

In [11]:
linear = SVR(kernel="linear")
rfe_linear = RFECV(linear, step=5, cv=5)
train_linear = rfe_linear.fit_transform(train, y_train)
print(train_linear.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(1212, 40)


Test SVR

In [12]:
train_x, val_x, train_y, val_y = train_test_split(train_linear, y_train, test_size=0.2, random_state=36)
linear = SVR(kernel="linear")
clf = linear.fit(train_x, train_y)
print(clf.score(val_x, val_y)) #already correct r2 metric :)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


NameError: ignored

In [13]:
print(clf.score(val_x, val_y)) #already correct r2 metric :)

0.5074965285156073


In [ ]:
from IPython.display import display, Markdown